# DHS grouping
Does further preprocessing steps after the unified columns have been loaded from DHS_loading_from_preprocessing.ipynb
Especially:
- loading locations file and merging them
- groups on either GEID (a survey ID) and adm2 or GEID and cluster ID
- creates DHS dataset

In [119]:
from icecream import ic
import scipy.stats
import pandas as pd
import numpy as np
from warnings import warn

In [120]:
# folder_path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
# 'BR' (birth R), 'HW!' (WHO height/weight standards children), 'IR'/'PR'? (Individual R, hh member R,), 'KR' (children R), 'AR' (HIV R)

urban_rural_all_mode = 'R' # 'all', 'U', 'R'
# 'adm2_gaul' or 'cluster number' (possibly 'adm1_gaul' etc.); additionally to survey ID
group_by_cols = ['Meta; GEID_init', 'Meta; cluster number']
preprocessing_f = f"{input_dir}Manually_adjusted_df_V3_{dataset_type}.pkl"
# unify_columns_f = f"{input_dir}summary_df_V3_{dataset_type}_unify_cols_reviewed.csv"
locations_f = input_dir + 'locations.csv'
# numeric_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_numeric{add_str}.pkl"
out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_cols[1].split('; ')[1]}_{urban_rural_all_mode}.pkl"
meta_data_cols_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_cols[1].split('; ')[1]}_meta_data_cols_{urban_rural_all_mode}.pkl"
### to do: urban/rural mode
### 2: drop binary, urban/rural, 2nd column -> in loading, drop binary, urban/rural, 2nd column
#to do: this should be created after grouping!? Maybe it makes sense even before grouping??? 
# -> yes prolly, as it increases the continuity of the distribution (more HH than adm2) (these are not the same if compared for urban/rural modes)
one_hot_cols = ['adm0_gaul']
frequency_hot_cols = ['adm1_gaul', 'adm2_gaul']
# cut_perc = 10

In [121]:
locations_df = pd.read_csv(locations_f)
print(locations_df.columns.to_list())
locations_df = locations_df[['GEID', 'TIF_name', 'adm2_name', 'adm1_name', 'adm0_name', 'DHSCLUST', 'URBAN_RURA']]
locations_df = locations_df.rename(columns={'GEID': 'GEID_init'})
locations_df = locations_df.rename(columns={'TIF_name': 'GEID', 'adm2_name': 'adm2_gaul', 'adm1_name': 'adm1_gaul', 'adm0_name': 'adm0_gaul', 'DHSCLUST': 'DHSCLUST'})
locations_df

['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'Shape_Area']


,GEID_init,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AOGE52FL,AOGE52FL00000001,Lobito,Benguela,Angola,1,U
1,AOGE52FL,AOGE52FL00000002,Lobito,Benguela,Angola,2,U
2,AOGE52FL,AOGE52FL00000003,Benguela,Benguela,Angola,3,U
3,AOGE52FL,AOGE52FL00000004,Sumbe,Cuanza Sul,Angola,4,U
4,AOGE52FL,AOGE52FL00000005,Dande,Bengo,Angola,5,U
...,...,...,...,...,...,...,...
70751,ZWGE72FL,ZWGE72FL00000396,Harare,Harare,Zimbabwe,396,U
70752,ZWGE72FL,ZWGE72FL00000397,Makoni,Manicaland,Zimbabwe,397,R
70753,ZWGE72FL,ZWGE72FL00000398,Hurungwe,Mashonaland West,Zimbabwe,398,R
70754,ZWGE72FL,ZWGE72FL00000399,Harare,Harare,Zimbabwe,399,U


In [122]:
df = pd.read_pickle(preprocessing_f)
print(len(df['generated clnr from case id'].dropna()), len(df['generated hhnr from case id'].dropna()))
#these are empty from steps before
df = df.drop(columns=['generated clnr from case id', 'generated hhnr from case id'])

12110 84200


In [123]:
df = df.merge(locations_df, on='GEID', how='inner')
df[df['cluster number'] != df['DHSCLUST']][['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'DHSCLUST', 'cluster number']]

,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,cluster number
455349,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455350,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455351,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455352,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455353,GHGE33FL00000082,Agona West,Central,Ghana,81,82
...,...,...,...,...,...,...
1479154,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479155,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479156,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479157,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132


In [124]:
df[['type of place of residence', 'URBAN_RURA']].value_counts(normalize=True, dropna=False) * 100


type of place of residence  URBAN_RURA
rural                       R             65.062716
urban                       U             34.874341
rural                       U              0.050168
urban                       R              0.012775
Name: proportion, dtype: float64

In [125]:
df2 = df[~df['cluster number'] != df['DHSCLUST']]
print(len(df))
df2

1612581


,country code and phase,GEID,DHSID,DHSID + HHID,version_nr,subversion_nr,cluster number,household number,number of household members,month of interview,...,owns chickens/poultry,country code,share toilet with other households: applicable,number of households sharing toilet: applicable,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000001,5,1,1,1,3.0,12.0,...,NaN,AO,applicable,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
1,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000002,5,1,1,2,3.0,12.0,...,NaN,AO,applicable,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
2,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000003,5,1,1,3,3.0,12.0,...,NaN,AO,applicable,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
3,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000004,5,1,1,4,5.0,12.0,...,NaN,AO,applicable,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
4,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000005,5,1,1,5,3.0,12.0,...,NaN,AO,applicable,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612576,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000025,7,2,400,25,5.0,11.0,...,31.0,ZW,applicable,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612577,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000026,7,2,400,26,2.0,11.0,...,10.0,ZW,not applicable,not applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612578,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000027,7,2,400,27,5.0,11.0,...,0,ZW,applicable,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612579,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000028,7,2,400,28,8.0,11.0,...,5.0,ZW,applicable,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R


In [126]:
### Splitting numeric and string columns as well as meta data columns
df = df2

df = df[(df['year of interview'].notna())
        & (df['month of interview'].notna()) & (df['day of interview'].notna())]
# Initialize empty lists to hold the names of numeric and categorical columns
sub_df = df[['GEID', 'household number'] + [c[6:] for c in group_by_cols]]
sub_df.rename(columns={c: 'Meta; ' + c for c in sub_df.columns}, inplace=True)
numeric_columns = [sub_df.copy()]
categorical_columns = [sub_df.copy()]
meta_data_cols = ['country code and phase', 'GEID', 'DHSID', 'DHSID + HHID', 'version_nr', 'subversion_nr', 'household number', 'case identification', 'DHSCLUST', 'country code', 
       'adm1_gaul', 'adm0_gaul', 'GEID_init', 'month of interview',
       'year of interview', 'day of interview', 'cluster number', 'adm2_gaul', 'URBAN_RURA'
    #    'generated hhnr from case id',
    #    'generated clnr from case id', 
       ]
for group_by_col in group_by_cols:
    if group_by_col not in meta_data_cols and group_by_col[6:] not in meta_data_cols:
        warn(f'group_by_col {group_by_col} not in meta_data_cols, may result in errors!')
        meta_data_cols.append(group_by_col)
    
# Iterate over each column
for col in df.columns:
    if col in meta_data_cols:
        continue
    # elif ' applicable' in col:
    #     # meta_data_cols.append(col)
    #     # one_hot_cols.append(col)
    #     continue
    found_nr = False
    # Create a new column for the numeric values
    nrcol = pd.to_numeric(df[col], errors='coerce')
    if nrcol.notna().sum() > 0:
        found_nr = True
        #remove integer artefacts
        if col not in ['type of cooking fuel', 'language of interview', 'owns land usable for agriculture']:
            #rename col
            nrcol.name = 'DHS Num; ' + col
            numeric_columns.append(nrcol)
    # Create a new column for the non-numeric values
    str_col = df[col].where(nrcol.isna())
    if str_col.notna().sum() > 0:
        #only "don't know" or similar
        if col != 'number of rooms used for sleeping' and col != 'hectares of agricultural land (1 decimal)':
            #rename col
            str_col.name = 'DHS Cat; ' + col
            categorical_columns.append(str_col)
        if found_nr:
            print('found both', col)
            print(df[col].value_counts(dropna=False, normalize=True) * 100)
            print(str_col.value_counts(dropna=False, normalize=True) * 100)


/tmp/ipykernel_1695625/3749518231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df.rename(columns={c: 'Meta; ' + c for c in sub_df.columns}, inplace=True)


found both language of interview
language of interview
NaN         44.564530
english      5.852616
chichewa     4.643598
hausa        3.561338
french       2.926806
              ...    
arabic       0.000062
43.0         0.000062
85.0         0.000062
sherbro      0.000062
yalunka      0.000062
Name: proportion, Length: 239, dtype: float64
DHS Cat; language of interview
NaN                   44.934149
english                5.852616
chichewa               4.643598
hausa                  3.561338
french                 2.926806
                        ...    
arabic                 0.000062
yalunka                0.000062
tamacheck/bella        0.000062
hararigna/adirigna     0.000062
sherbro                0.000062
Name: proportion, Length: 212, dtype: float64
found both type of cooking fuel
type of cooking fuel
wood                       49.062871
charcoal                   14.069262
firewood, straw            10.082370
NaN                         9.608978
lpg                        

In [127]:
### Create meta data, numeric and categorical dfs
meta_data_df = df[meta_data_cols].copy()
# Create numeric df
numeric_df = pd.concat(numeric_columns, axis=1)

# Create categorical df
categorical_df = pd.concat(categorical_columns, axis=1)

if urban_rural_all_mode == 'all':
    pass
elif urban_rural_all_mode == 'U':
    meta_data_df = meta_data_df[meta_data_df['URBAN_RURA'] == 'U']
    # Adjust numeric and categorical dfs to match the index of meta_data_df
    numeric_df = numeric_df.loc[meta_data_df.index]
    categorical_df = categorical_df.loc[meta_data_df.index]
elif urban_rural_all_mode == 'R':
    meta_data_df = meta_data_df[meta_data_df['URBAN_RURA'] == 'R']
    # Adjust numeric and categorical dfs to match the index of meta_data_df
    numeric_df = numeric_df.loc[meta_data_df.index]
    categorical_df = categorical_df.loc[meta_data_df.index]


### Generate date and co
print('cluster number' in meta_data_df.columns, 'GEID_init' in meta_data_df.columns)
print([c for c in meta_data_df.columns if c in ['cluster number', 'GEID_init']])
# meta_data_df['month of interview'] = meta_data_df['month of interview'].fillna(6)
# meta_data_df['day of interview'] = meta_data_df['day of interview'].fillna(15)

meta_data_df['year'] = meta_data_df['year of interview'].astype(float).astype('Int32')
meta_data_df['month'] = meta_data_df['month of interview'].astype(float).astype('Int32')
meta_data_df['day'] = meta_data_df['day of interview'].astype(float).astype('Int32')

#replace days >28 with 28
meta_data_df.loc[meta_data_df['day'] > 28, 'day'] = 28

# Now you can combine the year, month, and day columns into a single date column
meta_data_df['date'] = pd.to_datetime(meta_data_df[['year', 'month', 'day']])

# Group the DataFrame by 'GEID_init' and 'adm2_gaul'/'cluster number'
grouped = meta_data_df.groupby([c[6:] for c in group_by_cols])

# Calculate the median date for each group and reset the index
median_date = grouped['date'].median().reset_index()

# Rename the date column in median_date
median_date.rename(columns={'date': 'median_date'}, inplace=True)

# Merge meta_data_df with median_date
meta_data_df = pd.merge(meta_data_df, median_date, on=[c[6:] for c in group_by_cols], how='left')

# Convert the median date to separate year, month, and day columns
meta_data_df['year'] = meta_data_df['median_date'].dt.year.astype('Int32')
meta_data_df['month'] = meta_data_df['median_date'].dt.month.astype('Int32')

meta_data_df[['year', 'month', 'year of interview', 'month of interview', 'day of interview', 'date', 'median_date']]


True True
['GEID_init', 'cluster number']


,year,month,year of interview,month of interview,day of interview,date,median_date
0,2006,11,2006.0,11.0,27.0,2006-11-27,2006-11-27
1,2006,11,2006.0,11.0,29.0,2006-11-28,2006-11-27
2,2006,11,2006.0,11.0,28.0,2006-11-28,2006-11-27
3,2006,11,2006.0,11.0,26.0,2006-11-26,2006-11-27
4,2006,11,2006.0,11.0,27.0,2006-11-27,2006-11-27
...,...,...,...,...,...,...,...
1045333,2015,11,2015.0,11.0,10.0,2015-11-10,2015-11-10
1045334,2015,11,2015.0,11.0,12.0,2015-11-12,2015-11-10
1045335,2015,11,2015.0,11.0,12.0,2015-11-12,2015-11-10
1045336,2015,11,2015.0,11.0,10.0,2015-11-10,2015-11-10


In [128]:
# Drop the original date column
meta_data_df['date'] = meta_data_df['median_date']
meta_data_df.drop(columns=['day', 'year of interview', 'month of interview', 'day of interview', 'median_date'], inplace=True)

def rreplace(s, old, new, occurrence=1):
    li = s.rsplit(old, occurrence)
    return new.join(li)

### to do: urban/rural mode
#to do: this should be created after grouping!? Maybe it makes sense even before grouping??? 
# -> yes prolly, as it increases the continuity of the distribution (more HH than adm2) (these are not comparable if compared for urban/rural modes)
for col in one_hot_cols:
    print(col)
    # Convert the column to a one-hot encoded DataFrame
    one_hot = pd.get_dummies(meta_data_df[col], prefix=col)

    # Replace '_' with ': ' in column names
    one_hot.columns = ['Meta one-hot encoding; ' + rreplace(col, '_', ': ', 1) for col in one_hot.columns]

    # Add the one-hot encoded columns to meta_data_df
    meta_data_df = pd.concat([meta_data_df, one_hot], axis=1)
    
for col in frequency_hot_cols:
    # Calculate the frequency of each category in the column
    freq = meta_data_df[col].value_counts(normalize=True)

    # Map the frequencies to the original DataFrame
    meta_data_df['Meta frequency encoding; ' + col] = meta_data_df[col].map(freq)

# rename columns
meta_data_df.rename(columns={c: 'Meta; ' + c for c in meta_data_df.columns if c[:4] != 'Meta'}, inplace=True)

adm0_gaul


In [129]:
for col in categorical_df.columns:
    print(categorical_df[col].dtype)
    print(categorical_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
Meta; GEID
MWGE43FL00001002    0.006314
MWGE43FL00003258    0.006122
CIGE33FL00000205    0.005835
CIGE33FL00000182    0.005740
CIGE33FL00000204    0.005740
                      ...   
AOGE52FL00000012    0.000096
EGGE4BFL04130101    0.000096
CMGE71FL00000103    0.000096
EGGE4BFL02510301    0.000096
NIGE81FL00000009    0.000096
Name: proportion, Length: 42570, dtype: float64


int64
Meta; household number
1       1.974481
3       1.932007
5       1.912683
7       1.906943
8       1.895081
          ...   
8256    0.000096
8762    0.000096
8761    0.000096
2992    0.000096
6945    0.000096
Name: proportion, Length: 12368, dtype: float64


object
Meta; GEID_init
NGGE7BFL    2.259556
KEGE8AFL    2.255921
NGGE52FL    2.233345
NGGE6AFL    2.146865
KEGE71FL    2.138734
              ...   
MWGE7IFL    0.211798
MWGE71FL    0.209884
AOGE52FL    0.143494
MLGE63FL    0.105612
CIGE3BFL    0.067060
Name: proportion, Length: 155, dtype: float64


int64
Meta; cluster number
106        0.24633

In [130]:
for col in numeric_df.columns:
    print(numeric_df[col].dtype)
    print(numeric_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
Meta; GEID
MWGE43FL00001002    0.006314
MWGE43FL00003258    0.006122
CIGE33FL00000205    0.005835
CIGE33FL00000182    0.005740
CIGE33FL00000204    0.005740
                      ...   
AOGE52FL00000012    0.000096
EGGE4BFL04130101    0.000096
CMGE71FL00000103    0.000096
EGGE4BFL02510301    0.000096
NIGE81FL00000009    0.000096
Name: proportion, Length: 42570, dtype: float64


int64
Meta; household number
1       1.974481
3       1.932007
5       1.912683
7       1.906943
8       1.895081
          ...   
8256    0.000096
8762    0.000096
8761    0.000096
2992    0.000096
6945    0.000096
Name: proportion, Length: 12368, dtype: float64


object
Meta; GEID_init
NGGE7BFL    2.259556
KEGE8AFL    2.255921
NGGE52FL    2.233345
NGGE6AFL    2.146865
KEGE71FL    2.138734
              ...   
MWGE7IFL    0.211798
MWGE71FL    0.209884
AOGE52FL    0.143494
MLGE63FL    0.105612
CIGE3BFL    0.067060
Name: proportion, Length: 155, dtype: float64


int64
Meta; cluster number
106        0.24633

In [131]:
for col in meta_data_df.columns:
    # print(col)
    # print(meta_data_df[col])
    # print(meta_data_df[col].dtype)
    print(meta_data_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')
meta_data_df.to_csv(meta_data_cols_out_f.replace('.pkl', '.csv'), index=False)
meta_data_df.to_pickle(meta_data_cols_out_f)

Meta; country code and phase
NG6    2.953781
KE6    2.473171
MD7    2.358950
EG4    2.297152
NG7    2.259556
         ...   
BF3    0.261351
CM8    0.250254
GM7    0.246810
KM6    0.233609
TZ3    0.229973
Name: proportion, Length: 117, dtype: float64




Meta; GEID
MWGE43FL00001002    0.006314
MWGE43FL00003258    0.006122
CIGE33FL00000205    0.005835
CIGE33FL00000182    0.005740
CIGE33FL00000204    0.005740
                      ...   
AOGE52FL00000012    0.000096
EGGE4BFL04130101    0.000096
CMGE71FL00000103    0.000096
EGGE4BFL02510301    0.000096
NIGE81FL00000009    0.000096
Name: proportion, Length: 42570, dtype: float64


Meta; DHSID
MWHR41SV00001002    0.006314
MWHR41SV00003258    0.006122
CIHR35SV00000205    0.005835
CIHR35SV00000182    0.005740
CIHR35SV00000204    0.005740
                      ...   
AOHR51SV00000012    0.000096
EGHR4ASV04130101    0.000096
CMHR71SV00000103    0.000096
EGHR4ASV02510301    0.000096
NIHR82SV00000009    0.000096
Name: proportion, Length: 42570, dtype: float64


Meta; DHSID + HHID
AOHR51SV0000000700000001    0.000096
NGHR7BSV0000048800000041    0.000096
NGHR7BSV0000048800000019    0.000096
NGHR7BSV0000048800000020    0.000096
NGHR7BSV0000048800000022    0.000096
                              ...  

In [132]:
print(len(categorical_df), len(df), len(numeric_df), len(meta_data_df))
# print('GEID' in categorical_df.columns, 'GEID' in numeric_df.columns, 'GEID' in meta_data_df.columns)
# print('household number' in categorical_df.columns, 'household number' in numeric_df.columns, 'household number' in meta_data_df.columns)

1045338 1605436 1045338 1045338


In [133]:
cols = ['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'year', 'month', 'household number', 'cluster number', 'date', 'URBAN_RURA']
cols = ['Meta; ' + c for c in cols]

for c in group_by_cols:
    if c in cols:    
        cols.remove(c)

for col in meta_data_df.columns:
    if ' applicable' in col or 'one-hot encoding' in col or 'frequency encoding' in col:
        print(col)
        cols.append(col)
    
numeric_df2 = pd.merge(meta_data_df[cols], numeric_df, on=['Meta; GEID', 'Meta; household number'])

#only working when not dropping due to urban/rural mode
if urban_rural_all_mode == 'all':
    assert len(df) == len(numeric_df2) == len(meta_data_df)

for na, df in [('numeric', numeric_df2), ('cat', categorical_df)]:
    for col in df.columns:
        if '_x' in col:
            print(na, col)

numeric_df2

Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Malawi
Meta one-hot encoding; adm0_gaul: Mali
Meta one-hot encoding; adm0_gaul: Moro

,Meta; GEID,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; household number,Meta; date,Meta; URBAN_RURA,Meta one-hot encoding; adm0_gaul: Angola,...,DHS Num; number of eligible men in hh,DHS Num; number of children 5 and under (de jure),DHS Num; number of households sharing toilet,DHS Num; hectares of agricultural land (1 decimal),DHS Num; number of rooms used for sleeping,DHS Num; owns goats,DHS Num; owns horses/ donkeys/ mules,DHS Num; time to get to water source (minutes),DHS Num; owns cows/ bulls,DHS Num; owns chickens/poultry
0,AOGE52FL00000007,Baia Farta,Benguela,Angola,2006,11,1,2006-11-27,R,True,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,AOGE52FL00000007,Baia Farta,Benguela,Angola,2006,11,3,2006-11-27,R,True,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,AOGE52FL00000007,Baia Farta,Benguela,Angola,2006,11,6,2006-11-27,R,True,...,NaN,3.0,10.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,AOGE52FL00000007,Baia Farta,Benguela,Angola,2006,11,8,2006-11-27,R,True,...,NaN,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AOGE52FL00000007,Baia Farta,Benguela,Angola,2006,11,11,2006-11-27,R,True,...,NaN,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045333,ZWGE72FL00000400,Murehwa,Mashonaland East,Zimbabwe,2015,11,25,2015-11-10,R,False,...,NaN,1.0,0.0,0.0,2.0,7.0,0.0,0.0,NaN,31.0
1045334,ZWGE72FL00000400,Murehwa,Mashonaland East,Zimbabwe,2015,11,26,2015-11-10,R,False,...,NaN,0.0,10.0,30.0,2.0,0.0,0.0,0.0,NaN,10.0
1045335,ZWGE72FL00000400,Murehwa,Mashonaland East,Zimbabwe,2015,11,27,2015-11-10,R,False,...,NaN,2.0,0.0,10.0,3.0,3.0,0.0,10.0,NaN,0.0
1045336,ZWGE72FL00000400,Murehwa,Mashonaland East,Zimbabwe,2015,11,28,2015-11-10,R,False,...,NaN,0.0,0.0,12.0,3.0,8.0,0.0,0.0,NaN,5.0


In [134]:
for col in numeric_df2:
    print(col)

Meta; GEID
Meta; adm2_gaul
Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; household number
Meta; date
Meta; URBAN_RURA
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_ga

In [135]:
# ### Aggregating the data: (categorical values are aggregated further below)
# # mean, median, std, skewness, kurtosis, percentage of valid answers for numerical columns
# # mean for one-hot and frequency encoding columns //should be the same as .agg('first')?!
# # first for meta data columns except encoding columns
# numeric_df = numeric_df2

# # # List of columns to exclude from the aggregation
# # cols_to_exclude = ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'GEID_init', 'adm2_gaul', 'GEID', 'household number', 'cluster number']
# # cols_to_exclude = ['Meta; ' + c for c in cols_to_exclude]
# # # if 'cluster number' != group_by_col:
# # #     cols_to_exclude.append('cluster number')
# cols_to_include_again = ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'date']
# cols_to_include_again = ['Meta; ' + c for c in cols_to_include_again]
# # cols_to_include_again += [c for c in numeric_df.columns if 'Meta one-hot encoding;' in c or 'Meta frequency encoding;' in c or 'Meta DHS applicable;' in c]
# if 'Meta; adm2_gaul' != group_by_cols[1]:
#     cols_to_include_again.append('Meta; adm2_gaul')

# # List of columns to include in the aggregation
# cols_to_include = [col for col in numeric_df.columns if 'DHS Num;' in col]
# cols_for_mean = [col for col in numeric_df.columns if 'Meta one-hot encoding;' in col or 'Meta frequency encoding;' in col or 'Meta DHS applicable;' in col]
# print(cols_to_include_again)
# print(cols_for_mean)
    
# # Group the DataFrame
# numeric_df = numeric_df.dropna(subset=group_by_cols).iloc[:1000, :]
# grouped_df = numeric_df.groupby(group_by_cols)
    
# # Perform the aggregation on a copy of the DataFrame
# # agg_df = grouped_df[cols_to_include].agg(['mean', 'median', 'std', lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan])
# agg_df = grouped_df[cols_to_include].agg(['mean', 'median', 'std', 
#                                           lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan,
#                                           lambda x: scipy.stats.kurtosis(x.dropna()) if not x.dropna().empty else np.nan])

# mean_df = grouped_df[cols_for_mean].agg('mean')
# # Get the first occurrence of the cols_to_exclude in each group
# meta_df = grouped_df[cols_to_include_again].agg('first')
# # meta_df = grouped_df[cols_to_exclude].first()
# # Add a second level to the columns in grouped_df
# meta_df.columns = pd.MultiIndex.from_product([meta_df.columns, ['']])
# print('mean df cols', mean_df.columns)
# # mean_df.columns = pd.MultiIndex.from_product([mean_df.columns, ['']])
# mean_df.columns = pd.MultiIndex.from_product([mean_df.columns.astype(str).tolist(), ['']])
# print(meta_df.columns)
# print(mean_df.columns)
# print(agg_df.columns)

# print(len(grouped_df), len(agg_df), len(mean_df), len(meta_df))
# assert len(grouped_df) == len(agg_df) == len(mean_df) == len(meta_df)
# # Merge the aggregation result back to the original DataFrame
# grouped_df = pd.concat([meta_df, mean_df, agg_df], axis=1)

# # Rename the last two aggregation functions to 'skewness' and 'kurtosis'
# grouped_df.rename(columns={'<lambda_0>': 'skewness', '<lambda_1>': 'kurtosis'}, level=1, inplace=True)
# # grouped_df.rename(columns={'<lambda_0>': 'skewness'}, level=1, inplace=True)

# # Calculate the count of non-NaN values in each column
# count_df = numeric_df.groupby(group_by_cols).count()

# # Calculate the size of each group
# households = numeric_df.groupby(group_by_cols).size()

# assert len(grouped_df) == len(households)

# # Add the size of each group to the grouped DataFrame
# grouped_df[('Meta; amount of households', '')] = households.values

# # Calculate the percentage of valid answers
# percentage_df = count_df.div(grouped_df[('Meta; amount of households', '')], axis=0) * 100

# assert len(percentage_df) == len(grouped_df)
# # Add the percentage of valid answers to the grouped DataFrame
# for col in percentage_df.columns:
#     grouped_df[('Meta; ' + col, 'percentage of valid answers')] = percentage_df[col]

# # Flatten the MultiIndex
# grouped_df.columns = [': '.join(col).strip() if isinstance(col, tuple) and col[1] else col[0] for col in grouped_df.columns.values]

# #remove unnessarily produced columns
# drop_cols = []
# for col in grouped_df.columns:
#     if 'Meta' == col[:4] and 'percentage of valid answers' in col:
#         print(col)
#         drop_cols.append(col)
# grouped_df = grouped_df.drop(columns=drop_cols)

# numerical_grouped_df = grouped_df
# grouped_df

In [136]:
### Aggregating the data: (categorical values are aggregated further below)
# mean, median, std, skewness, kurtosis, percentage of valid answers for numerical columns
# mean for one-hot and frequency encoding columns //should be the same as .agg('first')?!
# first for meta data columns except encoding columns
numeric_df = numeric_df2

# Define the columns to include in each type of aggregation
cols_for_stats = [col for col in numeric_df.columns if 'DHS Num;' in col]
cols_for_mean = [col for col in numeric_df.columns if 'Meta one-hot encoding;' in col or 'Meta frequency encoding;' in col or 'Meta DHS applicable;' in col]
cols_for_first = ['Meta; ' + c for c in ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'date']]
if 'Meta; adm2_gaul' != group_by_cols[1]:
    cols_for_first.append('Meta; adm2_gaul')

# Group the DataFrame
grouped = numeric_df.groupby(group_by_cols)

# Perform the aggregations
agg_df = grouped[cols_for_stats].agg(['mean', 'median', 'std', 
                                      lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan,
                                      lambda x: scipy.stats.kurtosis(x.dropna()) if not x.dropna().empty else np.nan])
agg_df.rename(columns={'<lambda_0>': 'skewness', '<lambda_1>': 'kurtosis'}, level=1, inplace=True)


mean_df = grouped[cols_for_mean].agg('mean')
first_df = grouped[cols_for_first].agg('first')

# Rename the columns of agg_df
agg_df.columns = [f'{col[0]}: {col[1]}' for col in agg_df.columns]

# Combine the results into a single DataFrame
result = pd.concat([first_df, mean_df, agg_df], axis=1)

# Calculate the count of non-NaN values and the size of each group
count_df = grouped[cols_for_stats].count()
group_sizes = grouped.size()

# Calculate the percentage of valid answers
percentage_df = count_df.div(group_sizes, axis=0) * 100

# Add the group sizes and percentages to the result DataFrame
result['Meta; amount of households'] = group_sizes
for col in percentage_df.columns:
    result[f'Meta; {col}: percentage of valid answers'] = percentage_df[col]

del numeric_df2, mean_df, first_df, agg_df, count_df, group_sizes, percentage_df, numeric_df, grouped

numerical_grouped_df = result
numerical_grouped_df

Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; cluster number                                     
AOGE52FL        7                             Benguela          Angola   
                8                             Benguela          Angola   
                9                             Benguela          Angola   
                10                            Benguela          Angola   
                11                          Cuanza Sul          Angola   
...                                                ...             ...   
ZWGE72FL        394                             Harare        Zimbabwe   
                395                   Mashonaland West        Zimbabwe   
                397                         Manicaland        Zimbabwe   
                398                   Mashonaland West        Zimbabwe   
                400                   Mashonaland East        Zimbabwe   

                                      Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; cluster number                                       
AOGE52FL        7                           2006           11 2006-11-27   
                8                           2006           12 2006-12-04   
                9                           2006           11 2006-11-25   
                10                          2006           12 2006-12-07   
                11                          2006           12 2006-12-11   
...                                          ...          ...        ...   
ZWGE72FL        394                         2015            8 2015-08-06   
                395                         2015            8 2015-08-28   
                397                         2015           10 2015-10-27   
                398                         2015            9 2015-09-28   
                400                         2015           11 2015-11-10   

                                     Meta; adm2_gaul  \
Meta; GEID_init Meta; cluster number                   
AOGE52FL        7                         Baia Farta   
                8                             Lobito   
                9                              Cubal   
                10                           Balombo   
                11                            Amboim   
...                                              ...   
ZWGE72FL        394                           Harare   
                395                         Hurungwe   
                397                           Makoni   
                398                         Hurungwe   
                400                          Murehwa   

                                      Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; cluster number                                             
AOGE52FL        7                                                          1.0   
                8                                                          1.0   
                9                                                          1.0   
                10                                                         1.0   
                11                                                         1.0   
...                                                                        ...   
ZWGE72FL        394                                                        0.0   
                395                                                        0.0   
                397                                                        0.0   
                398                                                        0.0   
                400                                                        0.0   

                                      Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; cluster number                                            
AOGE52FL        7                                                         0.0   
                8                               

In [137]:
for col in numerical_grouped_df:
    print(col)

Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; date
Meta; adm2_gaul
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Ma

In [138]:
### Create categorical grouped df
# merge with meta data to ensure only valid rows are used in urban/rural mode
grouped_dfs = []

assert len(categorical_df) == len(meta_data_df)
# categorical_df = pd.merge(meta_data_df[group_by_cols + ['Meta; URBAN_RURA']], categorical_df, on=group_by_cols, how='inner')
categorical_df['DHS Cat; URBAN_RURA'] = meta_data_df['Meta; URBAN_RURA']
# categorical_df = categorical_df.drop(columns=['Meta; URBAN_RURA'])

# Group the DataFrame by 'GEID_init' and 'adm2_gaul'
grouped = categorical_df.groupby(group_by_cols)

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def process_column(col):
    if col not in sub_df.columns:
        # Calculate the normalized count of each category excluding NaNs
        counts = grouped[col].value_counts(normalize=True).unstack()
        
        # Only fill NaNs with 0 if there were values in the group
        if not counts.empty:
            counts = counts.fillna(0)
        
        # Calculate the normalized count of NaNs
        nan_counts = grouped[col].apply(lambda x: x.isna().mean()).rename('NaN')
        
        # Append the count of NaNs to the counts of the other categories
        counts = pd.concat([counts, nan_counts], axis=1)
        
        # Flatten the MultiIndex and add a ': ' between the original column name and the value_counts.index items
        counts.columns = [f'{col}: {c}' for c in counts.columns]
        
        return counts
    
    
    
# Create a pool of processes
with ProcessPoolExecutor(max_workers=4) as executor:
    # Use tqdm for progress bar
    grouped_dfs = list(tqdm(executor.map(process_column, categorical_df.columns), total=len(categorical_df.columns)))

# Concatenate the results
grouped_df = pd.concat(grouped_dfs, axis=1)
categorical_grouped_df = grouped_df


# for col in categorical_df.columns:
#     if col not in sub_df.columns:
#         # Calculate the normalized count of each category excluding NaNs
#         counts = grouped[col].value_counts(normalize=True).unstack()
        
#         # Only fill NaNs with 0 if there were values in the group
#         if not counts.empty:
#             counts = counts.fillna(0)
        
#         # Calculate the normalized count of NaNs
#         nan_counts = grouped[col].apply(lambda x: x.isna().mean()).rename('NaN')
        
#         # Append the count of NaNs to the counts of the other categories
#         counts = pd.concat([counts, nan_counts], axis=1)
        
#         # Flatten the MultiIndex and add a ': ' between the original column name and the value_counts.index items
#         counts.columns = [f'{col}: {c}' for c in counts.columns]
        
#         grouped_dfs.append(counts)
        
# # Concatenate the results
# grouped_df = pd.concat(grouped_dfs, axis=1)
# # grouped_df.columns = ['DHS Categorical; ' + col for col in grouped_df.columns]

# grouped_df.reset_index(inplace=True)
# categorical_grouped_df = grouped_df[[c for c in grouped_df.columns if c not in numerical_grouped_df.columns or c in ['GEID_init', group_by_col]]]
# grouped_df[[c for c in grouped_df.columns if c in numerical_grouped_df.columns]]

100%|██████████| 68/68 [01:08<00:00,  1.01s/it]


In [139]:
# merge both dfs
grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')
grouped_df

/tmp/ipykernel_1695625/1384408156.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')
/tmp/ipykernel_1695625/1384408156.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')


Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; cluster number                                     
AOGE52FL        7                             Benguela          Angola   
                8                             Benguela          Angola   
                9                             Benguela          Angola   
                10                            Benguela          Angola   
                11                          Cuanza Sul          Angola   
...                                                ...             ...   
ZWGE72FL        394                             Harare        Zimbabwe   
                395                   Mashonaland West        Zimbabwe   
                397                         Manicaland        Zimbabwe   
                398                   Mashonaland West        Zimbabwe   
                400                   Mashonaland East        Zimbabwe   

                                      Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; cluster number                                       
AOGE52FL        7                           2006           11 2006-11-27   
                8                           2006           12 2006-12-04   
                9                           2006           11 2006-11-25   
                10                          2006           12 2006-12-07   
                11                          2006           12 2006-12-11   
...                                          ...          ...        ...   
ZWGE72FL        394                         2015            8 2015-08-06   
                395                         2015            8 2015-08-28   
                397                         2015           10 2015-10-27   
                398                         2015            9 2015-09-28   
                400                         2015           11 2015-11-10   

                                     Meta; adm2_gaul  \
Meta; GEID_init Meta; cluster number                   
AOGE52FL        7                         Baia Farta   
                8                             Lobito   
                9                              Cubal   
                10                           Balombo   
                11                            Amboim   
...                                              ...   
ZWGE72FL        394                           Harare   
                395                         Hurungwe   
                397                           Makoni   
                398                         Hurungwe   
                400                          Murehwa   

                                      Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; cluster number                                             
AOGE52FL        7                                                          1.0   
                8                                                          1.0   
                9                                                          1.0   
                10                                                         1.0   
                11                                                         1.0   
...                                                                        ...   
ZWGE72FL        394                                                        0.0   
                395                                                        0.0   
                397                                                        0.0   
                398                                                        0.0   
                400                                                        0.0   

                                      Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; cluster number                                            
AOGE52FL        7                                                         0.0   
                8                               

In [140]:
grouped_df.to_csv(out_f.replace('.pkl', '.csv'))
grouped_df.to_pickle(out_f)

In [141]:
for col in grouped_df.columns:
    print(col)
    # print(grouped_df[col].dtype)
    # print(grouped_df[col].value_counts(normalize=True, dropna=False) * 100)
    # print('\n')

Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; date
Meta; adm2_gaul
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Ma